In [3]:
#traces
import pandas as pd
import numpy as np
import re
import sys
import nbimporter
#for ordered dict for traces implementation
from collections import OrderedDict

import traces
from datetime import datetime
from datetime import timedelta
#import sources.endomondolib as endo
#import sources.pysparkconvenience as ps
from numpy import array
from math import sqrt
from pyspark.sql.types import DoubleType
#for timedelta manipulation
from math import fabs

from pyspark.sql.functions import *
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.sql import DataFrameReader
from pyspark.sql import DataFrame
from pyspark.sql.functions import count
from pyspark.sql.window import Window
from pyspark.sql import SQLContext
from IPython.display import display, HTML
from pyspark.sql.functions import col
from pyspark.mllib.linalg import Vectors
from pyspark.sql import HiveContext
from pyspark.ml.feature import VectorAssembler

# Disable warnings, set Matplotlib inline plotting and load Pandas package
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
pd.options.display.mpl_style = 'default'



from pyspark import SparkContext
#sc =SparkContext()


pyspark_submit_args = os.environ.get("PYSPARK_SUBMIT_ARGS", "")
if not "pyspark-shell" in pyspark_submit_args: pyspark_submit_args += " pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

#%load_ext autotime
from pyspark.sql import SQLContext
#sqlContext = SQLContext(sc)

del min
del max

data_run='/Users/momori/dse/maomori/thesis/git/fitness_capstone/data/run_data_with_hr_spd.csv'
data_users='/Users/momori/dse/maomori/thesis/git/fitness_capstone/data/run_with_hr_users.csv'

sqlContext = HiveContext(sc)



In [4]:
#create pyspark dataframe from csv
def df_from_csv(csv_file):
    text = sc.textFile(csv_file)\
        .map(lambda line: line.split(","))
    #didn't work with take(1). believe returns 
    #different object then first()
    schema = text.first()
    data = text.filter(lambda x: x != schema)
    df = sqlContext.createDataFrame(data, schema)
    return df

def change_column_names(df, old_names, new_names):
    pass
    return df;

##input
##   string: 2013-02-22 21:38:45
##output: 
##   list: [u'2013', u'02', u'22', u'21', u'38', u'45']
def datetime_to_trace_time(timestamp):
    #strip double quotes
    timestamp = timestamp.replace('"', '')
    r_list = re.split('[- :]', timestamp)
    return [int(x) for x in r_list]

In [14]:

df_runs = df_from_csv(data_run)
print type(df_runs)
df_runs.show(2)
df_runs.count()

<class 'pyspark.sql.dataframe.DataFrame'>
+--------------------+----------+------------+-------------+-------------+------------+-----------+--------+
|              "time"|"altitude"|"heart_rate"|   "latitude"|  "longitude"|     "speed"|"workoutid"|    "id"|
+--------------------+----------+------------+-------------+-------------+------------+-----------+--------+
|"2013-02-22 21:38...|          |   147.00000|51.4394768234|-0.8953504544|6.7140000000|  167479013|13856798|
|"2013-02-22 21:38...|          |   148.00000|51.4392022323|-0.8952703234|9.3492000000|  167479013|13856799|
+--------------------+----------+------------+-------------+-------------+------------+-----------+--------+
only showing top 2 rows



300000

In [6]:
#get list of workoutids
#woutid_list = df_runs.select('"workoutid"').distinct()\
#    .rdd.map(lambda r:r[0]).collect()
#woutid_list

In [18]:
#class to create summary stats of workouts
class SummaryStat():
    data_points = ''
    
    #list of tuples. 
    #   key: column name
    #   value: metric (sum/avg/etc)
    summary_conf = ''
    partition_name = ''
    def __init__(self, sql_df, summary_conf = [], \
                partition_name = ''):
        if partition_name:
            self.partition_name = partition_name
        else:
            self.partition_name = '"workoutid"'
        self.data_points = sql_df
        if not isinstance(sql_df, DataFrame):
            print 'please enter spark dataframe object'
            return
        #if summary_conf not defined, default to sum for all columns
        if not summary_conf:
            col_names = self.data_points.schema.names
            for i in col_names:
                summary_conf.append((i, 'sum'))
        else:
            if not isinstance(summary_conf, list):
                print 'check summary_conf type'
                return
            for element in summary_conf:
                if not isinstance(element, dict):
                    print 'check summary_conf type 2'
                    return
            self.summary_conf = summary_conf
        
    def describe(self):
        print self.data_points.limit(2)

    def generate_summary(self):
        #windows function test
        r_df = ''
        r_list_cols = []
        for dic in self.summary_conf:
            col_name = dic.keys()[0] #should only have 1 key per dictionary
            aggregation_type = dic[col_name]
            w = Window.partitionBy(self.partition_name)
            if aggregation_type == 'avg':
                new_col_name = aggregation_type + '_' + col_name
                tmp = self.data_points.select(col_name, self.partition_name,\
                                        avg(col_name).over(w).alias(new_col_name)\
                                            )
                r_list_cols.append(tmp[[new_col_name, self.partition_name]].distinct())
            if aggregation_type == 'sum':
                new_col_name = aggregation_type + '_' + col_name
                tmp = self.data_points.select(col_name, self.partition_name,\
                                        sum(col_name).over(w).alias(new_col_name)\
                                            )
                r_list_cols.append(tmp[[new_col_name, self.partition_name]].distinct())                
        
        for df in r_list_cols:
            if not r_df:
                r_df = df
            else:
                r_df = r_df.join(df, (r_df[self.partition_name]==df[self.partition_name]), 'inner')\
                    .drop(df[self.partition_name])
        return r_df


In [19]:
sub_df = df_runs.limit(20)
sub_df.head(2)

[Row("time"=u'"2013-02-22 21:38:45"', "altitude"=u'', "heart_rate"=u'147.00000', "latitude"=u'51.4394768234', "longitude"=u'-0.8953504544', "speed"=u'6.7140000000', "workoutid"=u'167479013', "id"=u'13856798'),
 Row("time"=u'"2013-02-22 21:38:57"', "altitude"=u'', "heart_rate"=u'148.00000', "latitude"=u'51.4392022323', "longitude"=u'-0.8952703234', "speed"=u'9.3492000000', "workoutid"=u'167479013', "id"=u'13856799')]

In [21]:
#probably smarter to go from top to down so creating summary will
#be O(n), where n is number of rows in df
s_conf = [{'"heart_rate"': 'avg'}, {'"speed"': 'sum'}]

obj = SummaryStat(df_runs, summary_conf=s_conf)
#obj.describe()
sumstats = obj.generate_summary()
sumstats.show()
sumstats.count()

+------------------+------------------+-----------+
|  avg_"heart_rate"|       sum_"speed"|"workoutid"|
+------------------+------------------+-----------+
|            158.24| 4727.620800000004|  171117914|
|           148.138| 5788.652399999991|  219630823|
|           152.662|         3654.6444|  325622855|
|165.95172413793102|3477.5604000000003|  430188558|
|           164.334| 5517.320400000002|  325634772|
|           162.708| 4561.351200000003|  358388284|
|           144.008| 5381.747999999998|  513660945|
|           151.172| 6464.520000000001|  532228357|
|           148.964| 6392.789999999996|  628219304|
|127.00862068965517|         1019.1672|  436651654|
| 145.0976430976431|2982.8879999999967|  547204459|
|150.87179487179486|               0.0|  255210417|
|            147.46| 5964.300000000004|  472955225|
|118.47909967845659| 3017.051999999998|  498615552|
|            151.35| 7251.742800000004|  531303446|
| 147.9141914191419|         3128.1012|  557166841|
|123.8279069

714

In [10]:
sub_df.limit(2)


DataFrame["time": string, "altitude": string, "heart_rate": string, "latitude": string, "longitude": string, "speed": string, "workoutid": string, "id": string]

In [11]:
#windows function test
w = Window.partitionBy('"workoutid"')
tmp = sub_df.select('"heart_rate"', '"workoutid"', \
              sum('"heart_rate"').over(w).alias("sum")\
                )

tmp[['"heart_rate"', '"workoutid"', 'sum']].show()

+------------+-----------+------+
|"heart_rate"|"workoutid"|   sum|
+------------+-----------+------+
|    89.00000|  165504204| 515.0|
|    89.00000|  165504204| 515.0|
|   100.00000|  165504204| 515.0|
|   116.00000|  165504204| 515.0|
|   121.00000|  165504204| 515.0|
|   147.00000|  167479013|2251.0|
|   148.00000|  167479013|2251.0|
|   148.00000|  167479013|2251.0|
|   149.00000|  167479013|2251.0|
|   151.00000|  167479013|2251.0|
|   152.00000|  167479013|2251.0|
|   151.00000|  167479013|2251.0|
|   151.00000|  167479013|2251.0|
|   150.00000|  167479013|2251.0|
|   152.00000|  167479013|2251.0|
|   151.00000|  167479013|2251.0|
|   150.00000|  167479013|2251.0|
|   150.00000|  167479013|2251.0|
|   151.00000|  167479013|2251.0|
|   150.00000|  167479013|2251.0|
+------------+-----------+------+



In [12]:
#windows function test
w = Window.partitionBy('"workoutid"')
tmp = sub_df.select('"heart_rate"', '"speed"', '"workoutid"', \
              sum('"heart_rate"').over(w).alias("sum")\
              , avg('"speed"').over(w).alias("spd")\
                )

tmp[['"workoutid"', 'spd', 'sum']].distinct().show()

+-----------+--------+------+
|"workoutid"|     spd|   sum|
+-----------+--------+------+
|  165504204|     0.0| 515.0|
|  167479013|10.59552|2251.0|
+-----------+--------+------+



In [13]:
sub_df.show()

+--------------------+----------+------------+-------------+-------------+-------------+-----------+--------+
|              "time"|"altitude"|"heart_rate"|   "latitude"|  "longitude"|      "speed"|"workoutid"|    "id"|
+--------------------+----------+------------+-------------+-------------+-------------+-----------+--------+
|"2013-02-22 21:38...|          |   147.00000|51.4394768234|-0.8953504544| 6.7140000000|  167479013|13856798|
|"2013-02-22 21:38...|          |   148.00000|51.4392022323|-0.8952703234| 9.3492000000|  167479013|13856799|
|"2013-02-22 21:39...|          |   148.00000|51.4390821196|-0.8953502029|10.6308000000|  167479013|13856800|
|"2013-02-22 21:39...|          |   149.00000|51.4388703089|-0.8954045177|13.3740000000|  167479013|13856801|
|"2013-02-22 21:39...|          |   151.00000|51.4386848174|-0.8952745982|12.6468000000|  167479013|13856802|
|"2013-02-22 21:39...|          |   152.00000|51.4385818038|-0.8951149229|11.6316000000|  167479013|13856803|
|"2013-02-